# **기존 AAN_new_PCA+XGB와의 차이점**
- AAN_new 데이터셋을 리뉴얼함.
- 리뉴얼 데이터셋으로 PCA + XGB 이용
- PCA 0.98로 이용

# **리뉴얼된 AAN_new**
- float64 데이터를 float32로 바꿈.
- 에러를 삽입할 때, 각 섹터에 순차적으로 넣는대신 random하게 고름




In [7]:
!pip install ipython-autotime
%load_ext autotime
import pandas as pd
import numpy as np
import os
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
import pickle
import math

# read data pickle
path = os.getcwd() + "/drive/My Drive/data/AAN_new"

x_train = pd.read_pickle(path + '/reset_NoError_whole_x_train.pickle')
print('=== Data Loaded: reset_NoError_whole_x_train.pickle ===')
y_train = pd.read_pickle(path + '/reset_NoError_whole_y_train.pickle')
print('=== Data Loaded: reset_NoError_whole_y_train.pickle ===')
x_test = pd.read_pickle(path + '/reset_NoError_whole_x_test.pickle')
print('=== Data Loaded: reset_NoError_whole_x_test.pickle ===')
y_test = pd.read_pickle(path + '/reset_NoError_whole_y_test.pickle')
print('=== Data Loaded: reset_NoError_whole_y_test.pickle ===')



The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
=== Data Loaded: reset_NoError_whole_x_train.pickle ===
=== Data Loaded: reset_NoError_whole_y_train.pickle ===
=== Data Loaded: reset_NoError_whole_x_test.pickle ===
=== Data Loaded: reset_NoError_whole_y_test.pickle ===
time: 4.31 s


In [0]:
x_train

,time,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,2019-11-02 09:24:00+09:00,0.0311,1.0466,406.091705,1104.010986,32.0,0.0000,30.984301,35.038799,1.47800,0.2838,100.0,96.406502,55.853600,56.787102,171.890198,829.605225,6.713800,34.956799,305.595306,1101.636597,14.19640,14.0472,407.718994,92.108902,94.361900,12.0,12.2238,1.3338,0.20840,36.184700,37.115501,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8486,-4.0689,-1.9391,-0.3011,1.2569,13496.0,-224.0,70.0,10.0,27.929701,-0.0003,-0.007200,0.108500,0.0,256.184906,-0.0013,255.298706,259.530487,0.0,10027.0,27627.0,0,0,0,0,0
1,2019-11-02 09:24:01+09:00,0.0311,1.0471,406.091705,1104.010986,32.0,0.0000,35.950100,35.039902,1.48240,0.2838,100.0,96.490501,55.768002,56.794701,173.034698,830.909729,5.378700,28.005100,306.930511,1101.955444,14.21220,14.0472,407.718994,91.969002,94.361900,12.0,12.2238,1.3322,0.20460,36.184700,37.115501,28.021200,1.0,0.0,1.0,0.0,792.0,0.0,11.7642,-4.7702,-2.1559,-0.1027,1.6715,13512.0,-224.0,70.0,10.0,28.436100,0.0000,-0.007200,0.108500,0.0,256.564697,0.0001,257.559296,258.409210,0.0,10030.0,27627.0,0,0,0,0,0
2,2019-11-02 09:24:02+09:00,0.0311,1.0410,406.091705,1104.010986,32.0,0.0000,34.956799,35.024601,1.58160,0.2838,100.0,96.490501,55.874500,56.787102,170.936600,833.518982,7.667500,39.922100,305.404602,1104.187866,14.12440,14.0472,407.821198,91.835899,94.453400,12.0,12.2238,1.3330,0.20690,36.184700,37.115501,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6485,-4.6927,-1.8849,0.6518,1.9967,13528.0,-240.0,70.0,10.0,28.920700,0.0000,-0.007200,0.054200,0.0,257.360413,-0.0001,256.022186,258.770996,0.0,10026.0,27627.0,0,0,0,0,0
3,2019-11-02 09:24:03+09:00,0.0311,1.0430,406.091705,1104.010986,32.0,0.0000,28.005100,35.029598,1.38400,0.2838,100.0,96.490501,55.736599,56.794701,174.751297,828.735413,6.523100,33.963799,307.502686,1101.636597,14.20600,14.0472,407.667786,91.657204,94.369499,12.0,12.2238,1.3322,0.20460,36.184700,37.115501,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.6964,-3.9879,-1.5550,0.8548,1.7724,13544.0,-240.0,70.0,10.0,29.253500,0.0000,-0.007200,0.054200,0.0,257.107208,0.0007,256.148712,258.445404,0.0,10027.0,27627.0,0,0,0,0,0
4,2019-11-02 09:24:04+09:00,0.0311,1.0407,406.091705,1104.010986,32.0,0.0000,39.922100,35.023899,1.46590,0.2838,100.0,96.490501,55.745499,56.794701,171.508804,837.432922,6.523100,33.963799,307.312012,1105.144653,14.09210,14.0472,407.565613,91.525703,94.346603,12.0,12.2238,1.3322,0.20770,36.199902,37.100201,28.022100,1.0,0.0,1.0,0.0,793.0,0.0,11.8043,-3.7168,-1.6621,0.1528,1.3034,13560.0,-240.0,70.0,10.0,29.528400,-0.0003,-0.007200,0.054200,0.0,254.882797,-0.0013,255.606201,258.517792,0.0,10030.0,27627.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727193,2019-11-01 19:59:55+09:00,0.0998,1.3857,404.178711,1040.812500,32.0,9.5426,156.114197,35.714699,11.71167,11.1862,100.0,96.375999,51.506550,52.773998,157.108276,796.989502,29.363649,153.631363,270.118652,1042.957397,12.22482,11.3235,403.629608,77.613701,78.996300,12.0,12.2238,1.3887,0.17505,35.482800,36.703499,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7158,-3.8670,-1.5737,0.6780,1.5849,2888.0,12064.0,70.0,10.0,18.478701,0.0010,64.525497,54.922600,0.0,393.681305,-0.0031,329.029205,329.553589,0.0,10053.0,27617.0,0,0,0,0,0
727194,2019-11-01 19:59:56+09:00,0.

time: 163 ms


In [0]:
x_test

,time,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,2019-11-01 20:00:00+09:00,0.0998,1.3901,404.178711,1040.812500,32.0,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,0
1,2019-11-01 20:00:01+09:00,0.0998,1.3840,404.178711,1040.812500,32.0,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,0
2,2019-11-01 20:00:02+09:00,0.0998,1.3837,404.178711,1040.812500,32.0,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,0
3,2019-11-01 20:00:03+09:00,0.0998,1.3854,404.178711,1040.812500,32.0,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.3235,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,0
4,2019-11-01 20:00:04+09:00,0.0998,1.3839,404.178711,1040.812500,32.0,9.748800,149.162598,35.710098,12.209800,11.186200,100.0,96.375999,51.306599,52.773998,155.868500,794.815186,29.411301,157.107101,271.072388,1044.551880,12.079600,11.3235,403.578491,77.590897,78.996300,12.0,12.2162,1.3872,0.1787,35.482800,36.688202,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5318,-1.7957,0.8014,1.9603,1816.0,13136.0,70.0,10.0,18.225500,0.0007,65.263298,60.854301,0.0,390.136688,0.0007,324.688995,326.931396,0.0,10053.0,27616.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181795,2019-10-30 03:11:55+09:00,0.0305,1.3269,392.330292,1115.077148,32.0,84.305901,2556.180908,35.741001,40.301601,40.284698,0.0,-1.852400,58.626598,59.030201,185.050903,860.916077,1616.897583,2563.885010,312.271088,1113.117310,17.037399,16.8930,391.054810,42.979401,43.557701,12.0,11.6287,1.2697,0.7166,35.223400,35.543800,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.6964,-5.0314,-2.2791,0.0256,1.7013,7464.0,1712.0,70.0,10.0,27.112301,-0.0009,1.511900,9.403900,0.0,335.159912,0.0011,329.788788,331.072815,0.0,1002

time: 137 ms


In [0]:
x_train.dtypes

time          object
P1.B2004     float32
P1.B2016     float32
P1.B3004     float32
P1.B3005     float32
              ...   
attack         int64
attack.P1      int64
attack.P2      int64
attack.P3      int64
error          int64
Length: 65, dtype: object

time: 5.58 ms


In [8]:

num0=0 # 노말
num1=0 # 어택
for i in y_train["attack"]:
  if(i == 0):
    num0 = num0+1
  elif(i == 1):
    num1 = num1+1
print("<<train>>")
print('normal 수')
print(num0)
print('attack 수')
print(num1)
print("전체")
print(str(num0+num1))


num0=0 # 노말
num1=0 # 어택
for i in y_test["attack"]:
  if(i == 0):
    num0 = num0+1
  elif(i == 1):
    num1 = num1+1
print("")
print("<<test>>")
print('normal 수')
print(num0)
print('attack 수')
print(num1)
print("전체")
print(str(num0+num1))

<<train>>
normal 수
659695
attack 수
67503
전체
727198

<<test>>
normal 수
152888
attack 수
28912
전체
181800
time: 178 ms


In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# features to learn
features = ['P1.B2004', 'P1.B2016', 'P1.B3004', 'P1.B3005',\
'P1.B4002', 'P1.B4005', 'P1.B400B', 'P1.B4022', 'P1.FCV01D',\
'P1.FCV01Z', 'P1.FCV02D', 'P1.FCV02Z', 'P1.FCV03D', 'P1.FCV03Z',\
'P1.FT01', 'P1.FT01Z', 'P1.FT02', 'P1.FT02Z', 'P1.FT03',\
'P1.FT03Z', 'P1.LCV01D', 'P1.LCV01Z', 'P1.LIT01', 'P1.PCV01D',\
'P1.PCV01Z', 'P1.PCV02D', 'P1.PCV02Z', 'P1.PIT01', 'P1.PIT02',\
'P1.TIT01', 'P1.TIT02', 'P2.24Vdc', 'P2.Auto', 'P2.Emgy',\
'P2.On', 'P2.SD01', 'P2.SIT01', 'P2.TripEx', 'P2.VT01e',\
'P2.VXT02', 'P2.VXT03', 'P2.VYT02', 'P2.VYT03', 'P3.LCP01D', \
'P3.LCV01D', 'P3.LH', 'P3.LL', 'P3.LT01', 'P4.HT_FD',\
'P4.HT_LD', 'P4.HT_PO', 'P4.HT_PS', 'P4.LD', 'P4.ST_FD',\
'P4.ST_LD', 'P4.ST_PO', 'P4.ST_PS', 'P4.ST_PT01', 'P4.ST_TT01',
]

#scaling
scaler = StandardScaler()
scaler.fit(x_train[features])
scaled_x_train = scaler.transform(x_train[features])
scaled_x_test = scaler.transform(x_test[features])

# PCA 적용
pca_model = PCA(n_components=35)
pca_model.fit(scaled_x_train)
train_pca = pca_model.transform(scaled_x_train)
test_pca = pca_model.transform(scaled_x_test)

print("original x_train: {}".format(str(x_train[features].shape)))
print("transformed x_train: {}".format(str(train_pca.shape)))
print("original x_test: {}".format(str(x_test[features].shape)))
print("transformed x_test: {}".format(str(test_pca.shape)))

original x_train: (727198, 59)
transformed x_train: (727198, 35)
original x_test: (181800, 59)
transformed x_test: (181800, 35)
time: 6.67 s


In [10]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

# Xgboost 학습
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=2, learning_rate=0.1, 
                        max_depth=8, min_child_weight=10, gamma= 0, random_state=0)
clf.fit(train_pca, y_train, verbose =1)

y_pred = clf.predict(test_pca)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))


#save model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
pickle.dump(clf, open(path + filename, "wb"))


# attack & error distribution
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))

87.91914191419143 

confusion matrix
[[151300   1588]
 [ 20375   8537]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    152888
           1       0.84      0.30      0.44     28912

    accuracy                           0.88    181800
   macro avg       0.86      0.64      0.68    181800
weighted avg       0.88      0.88      0.85    181800

time: 361 ms


# **N**

# 5

In [12]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_5percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.68756875687569
confusion matrix
[[150878   2010]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    152888
           1       0.81      0.30      0.43     28912

    accuracy                           0.88    181800
   macro avg       0.85      0.64      0.68    181800
weighted avg       0.87      0.88      0.85    181800

error 갯수
7644
normal error인데 attack으로 분류된 경우
504
normal error인데 normal으로 분류된 경우
7140
오분류 에러: 6.593406593406594
time: 4.35 s


10

In [13]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_10percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.46369636963696
confusion matrix
[[150471   2417]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.93    152888
           1       0.78      0.30      0.43     28912

    accuracy                           0.87    181800
   macro avg       0.83      0.64      0.68    181800
weighted avg       0.86      0.87      0.85    181800

error 갯수
15288
normal error인데 attack으로 분류된 경우
1000
normal error인데 normal으로 분류된 경우
14288
오분류 에러: 6.541077969649398
time: 14.1 s


15

In [15]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_15percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.1974697469747
confusion matrix
[[149987   2901]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.93    152888
           1       0.75      0.30      0.42     28912

    accuracy                           0.87    181800
   macro avg       0.81      0.64      0.68    181800
weighted avg       0.86      0.87      0.85    181800

error 갯수
22933
normal error인데 attack으로 분류된 경우
1562
normal error인데 normal으로 분류된 경우
21371
오분류 에러: 6.811145510835913
time: 12 s


30

In [18]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/N/reset_Error_whole_x_test_30percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str((a/errors)*100)))




86.50935093509351
confusion matrix
[[148736   4152]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.97      0.92    152888
           1       0.67      0.30      0.41     28912

    accuracy                           0.87    181800
   macro avg       0.78      0.63      0.67    181800
weighted avg       0.85      0.87      0.84    181800

error 갯수
45866
normal error인데 attack으로 분류된 경우
3056
normal error인데 normal으로 분류된 경우
42810
오분류 에러: 6.662887541970086
time: 11.1 s


# **A1**

5

In [19]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_5percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




85.2040704070407
confusion matrix
[[146363   6525]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.92    152888
           1       0.57      0.30      0.39     28912

    accuracy                           0.85    181800
   macro avg       0.72      0.63      0.65    181800
weighted avg       0.83      0.85      0.83    181800

error 갯수
7644
normal error인데 attack으로 분류된 경우
5017
normal error인데 normal으로 분류된 경우
2627
오분류 에러: 65.63317634746207
time: 7.23 s


10

In [21]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_10percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




82.54400440044004
confusion matrix
[[141527  11361]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.93      0.90    152888
           1       0.43      0.30      0.35     28912

    accuracy                           0.83    181800
   macro avg       0.65      0.61      0.62    181800
weighted avg       0.80      0.83      0.81    181800

error 갯수
15288
normal error인데 attack으로 분류된 경우
9934
normal error인데 normal으로 분류된 경우
5354
오분류 에러: 64.97906855049712
time: 4.98 s


15

In [22]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_15percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




79.83608360836084
confusion matrix
[[136604  16284]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.89      0.88    152888
           1       0.34      0.30      0.32     28912

    accuracy                           0.80    181800
   macro avg       0.61      0.59      0.60    181800
weighted avg       0.79      0.80      0.79    181800

error 갯수
22933
normal error인데 attack으로 분류된 경우
14948
normal error인데 normal으로 분류된 경우
7985
오분류 에러: 65.18117995901103
time: 13.4 s


30

In [23]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1/reset_Error_whole_x_test_30percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




71.72002200220022
confusion matrix
[[121849  31039]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.86      0.80      0.83    152888
           1       0.22      0.30      0.25     28912

    accuracy                           0.72    181800
   macro avg       0.54      0.55      0.54    181800
weighted avg       0.75      0.72      0.73    181800

error 갯수
45866
normal error인데 attack으로 분류된 경우
29939
normal error인데 normal으로 분류된 경우
15927
오분류 에러: 65.27493132167619
time: 16.3 s


# **A2**

5

In [24]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_5percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.85258525852585
confusion matrix
[[151178   1710]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    152888
           1       0.83      0.30      0.44     28912

    accuracy                           0.88    181800
   macro avg       0.86      0.64      0.68    181800
weighted avg       0.87      0.88      0.85    181800

error 갯수
7644
normal error인데 attack으로 분류된 경우
204
normal error인데 normal으로 분류된 경우
7440
오분류 에러: 2.6687598116169546
time: 13.2 s


10

In [25]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_10percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.7981298129813
confusion matrix
[[151079   1809]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    152888
           1       0.83      0.30      0.43     28912

    accuracy                           0.88    181800
   macro avg       0.85      0.64      0.68    181800
weighted avg       0.87      0.88      0.85    181800

error 갯수
15288
normal error인데 attack으로 분류된 경우
367
normal error인데 normal으로 분류된 경우
14921
오분류 에러: 2.4005756148613293
time: 9.97 s


15

In [26]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_15percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.71727172717272
confusion matrix
[[150932   1956]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    152888
           1       0.81      0.30      0.43     28912

    accuracy                           0.88    181800
   macro avg       0.85      0.64      0.68    181800
weighted avg       0.87      0.88      0.85    181800

error 갯수
22933
normal error인데 attack으로 분류된 경우
596
normal error인데 normal으로 분류된 경우
22337
오분류 에러: 2.598874983648018
time: 10.1 s


30

In [27]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2/reset_Error_whole_x_test_30percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.54620462046205
confusion matrix
[[150621   2267]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    152888
           1       0.79      0.30      0.43     28912

    accuracy                           0.88    181800
   macro avg       0.84      0.64      0.68    181800
weighted avg       0.87      0.88      0.85    181800

error 갯수
45866
normal error인데 attack으로 분류된 경우
1155
normal error인데 normal으로 분류된 경우
44711
오분류 에러: 2.5182052064710243
time: 17 s


# **A1 N**

5

In [28]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_5percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




84.27337733773376
confusion matrix
[[144671   8217]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.95      0.91    152888
           1       0.51      0.30      0.37     28912

    accuracy                           0.84    181800
   macro avg       0.69      0.62      0.64    181800
weighted avg       0.82      0.84      0.82    181800

error 갯수
7644
normal error인데 attack으로 분류된 경우
6715
normal error인데 normal으로 분류된 경우
929
오분류 에러: 87.84667713239142
time: 6.42 s


10

In [29]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_10percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




80.55115511551155
confusion matrix
[[137904  14984]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.90      0.89    152888
           1       0.36      0.30      0.33     28912

    accuracy                           0.81    181800
   macro avg       0.62      0.60      0.61    181800
weighted avg       0.79      0.81      0.80    181800

error 갯수
15288
normal error인데 attack으로 분류된 경우
13544
normal error인데 normal으로 분류된 경우
1744
오분류 에러: 88.59236002093145
time: 11.1 s


15

In [30]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_15percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




76.92574257425743
confusion matrix
[[131313  21575]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.86    152888
           1       0.28      0.30      0.29     28912

    accuracy                           0.77    181800
   macro avg       0.57      0.58      0.58    181800
weighted avg       0.77      0.77      0.77    181800

error 갯수
22933
normal error인데 attack으로 분류된 경우
20215
normal error인데 normal으로 분류된 경우
2718
오분류 에러: 88.14808354772599
time: 13.6 s


30

In [31]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N/reset_Error_whole_x_test_30percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




65.93124312431243
confusion matrix
[[111325  41563]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.85      0.73      0.78    152888
           1       0.17      0.30      0.22     28912

    accuracy                           0.66    181800
   macro avg       0.51      0.51      0.50    181800
weighted avg       0.74      0.66      0.69    181800

error 갯수
45866
normal error인데 attack으로 분류된 경우
40465
normal error인데 normal으로 분류된 경우
5401
오분류 에러: 88.22439279640693
time: 10.7 s


# **A2 N**

5

In [32]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_5percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




87.06985698569856
confusion matrix
[[149755   3133]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.93    152888
           1       0.73      0.30      0.42     28912

    accuracy                           0.87    181800
   macro avg       0.81      0.64      0.67    181800
weighted avg       0.86      0.87      0.85    181800

error 갯수
7644
normal error인데 attack으로 분류된 경우
1621
normal error인데 normal으로 분류된 경우
6023
오분류 에러: 21.206174777603348
time: 8.08 s


10

In [33]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_10percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




86.24202420242024
confusion matrix
[[148250   4638]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.97      0.92    152888
           1       0.65      0.30      0.41     28912

    accuracy                           0.86    181800
   macro avg       0.76      0.63      0.66    181800
weighted avg       0.84      0.86      0.84    181800

error 갯수
15288
normal error인데 attack으로 분류된 경우
3215
normal error인데 normal으로 분류된 경우
12073
오분류 에러: 21.029565672422816
time: 10.6 s


15

In [34]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_15percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




85.35423542354236
confusion matrix
[[146636   6252]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.92    152888
           1       0.58      0.30      0.39     28912

    accuracy                           0.85    181800
   macro avg       0.73      0.63      0.65    181800
weighted avg       0.83      0.85      0.83    181800

error 갯수
22933
normal error인데 attack으로 분류된 경우
4903
normal error인데 normal으로 분류된 경우
18030
오분류 에러: 21.379671216151397
time: 9.75 s


30

In [35]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N/reset_Error_whole_x_test_30percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




82.79537953795379
confusion matrix
[[141984  10904]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.93      0.90    152888
           1       0.44      0.30      0.35     28912

    accuracy                           0.83    181800
   macro avg       0.66      0.61      0.63    181800
weighted avg       0.81      0.83      0.81    181800

error 갯수
45866
normal error인데 attack으로 분류된 경우
9777
normal error인데 normal으로 분류된 경우
36089
오분류 에러: 21.31644355295862
time: 13.7 s


# **A1 A2**

5

In [36]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_5percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




84.83223322332233
confusion matrix
[[145687   7201]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.95      0.91    152888
           1       0.54      0.30      0.38     28912

    accuracy                           0.85    181800
   macro avg       0.71      0.62      0.65    181800
weighted avg       0.82      0.85      0.83    181800

error 갯수
7644
normal error인데 attack으로 분류된 경우
5687
normal error인데 normal으로 분류된 경우
1957
오분류 에러: 74.39822082679225
time: 12.1 s


10

In [37]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_10percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




81.76952695269527
confusion matrix
[[140119  12769]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.92      0.89    152888
           1       0.40      0.30      0.34     28912

    accuracy                           0.82    181800
   macro avg       0.64      0.61      0.62    181800
weighted avg       0.80      0.82      0.81    181800

error 갯수
15288
normal error인데 attack으로 분류된 경우
11343
normal error인데 normal으로 분류된 경우
3945
오분류 에러: 74.19544740973313
time: 11.1 s


15

In [38]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_15percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




78.7106710671067
confusion matrix
[[134558  18330]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.88      0.87    152888
           1       0.32      0.30      0.31     28912

    accuracy                           0.79    181800
   macro avg       0.59      0.59      0.59    181800
weighted avg       0.78      0.79      0.78    181800

error 갯수
22933
normal error인데 attack으로 분류된 경우
16980
normal error인데 normal으로 분류된 경우
5953
오분류 에러: 74.0417738629922
time: 13.7 s


30

In [39]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2/reset_Error_whole_x_test_30percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




69.49339933993399
confusion matrix
[[117801  35087]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.85      0.77      0.81    152888
           1       0.20      0.30      0.24     28912

    accuracy                           0.69    181800
   macro avg       0.52      0.53      0.52    181800
weighted avg       0.75      0.69      0.72    181800

error 갯수
45866
normal error인데 attack으로 분류된 경우
33968
normal error인데 normal으로 분류된 경우
11898
오분류 에러: 74.05921597697642
time: 17.1 s


# **A1 A2 N**

5

In [40]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_5percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




84.53685368536854
confusion matrix
[[145150   7738]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.88      0.95      0.91    152888
           1       0.52      0.30      0.38     28912

    accuracy                           0.85    181800
   macro avg       0.70      0.62      0.64    181800
weighted avg       0.82      0.85      0.83    181800

error 갯수
7644
normal error인데 attack으로 분류된 경우
6228
normal error인데 normal으로 분류된 경우
1416
오분류 에러: 81.4756671899529
time: 9.67 s


10

In [41]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_10percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




81.13201320132013
confusion matrix
[[138960  13928]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.91      0.89    152888
           1       0.38      0.30      0.33     28912

    accuracy                           0.81    181800
   macro avg       0.63      0.60      0.61    181800
weighted avg       0.79      0.81      0.80    181800

error 갯수
15288
normal error인데 attack으로 분류된 경우
12497
normal error인데 normal으로 분류된 경우
2791
오분류 에러: 81.74385138670853
time: 7.58 s


15

In [42]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_15percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




77.77117711771177
confusion matrix
[[132850  20038]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.87      0.87      0.87    152888
           1       0.30      0.30      0.30     28912

    accuracy                           0.78    181800
   macro avg       0.58      0.58      0.58    181800
weighted avg       0.78      0.78      0.78    181800

error 갯수
22933
normal error인데 attack으로 분류된 경우
18695
normal error인데 normal으로 분류된 경우
4238
오분류 에러: 81.52008023372433
time: 8.64 s


30

In [43]:
picklename = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N/reset_Error_whole_x_test_30percent.pickle"
x_test = pd.read_pickle(picklename)

# 스케일링
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

print("")
acc = accuracy_score(y_test, y_pred)
print(acc*100)
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1
print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




67.65951595159517
confusion matrix
[[114467  38421]
 [ 20374   8538]]
              precision    recall  f1-score   support

           0       0.85      0.75      0.80    152888
           1       0.18      0.30      0.23     28912

    accuracy                           0.68    181800
   macro avg       0.52      0.52      0.51    181800
weighted avg       0.74      0.68      0.70    181800

error 갯수
45866
normal error인데 attack으로 분류된 경우
37309
normal error인데 normal으로 분류된 경우
8557
오분류 에러: 81.34347882963415
time: 15.7 s
